# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import pandas as pd
import logging
import json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [4]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        responses = []
        preds = scoring_func(inputString)
        responses.append(preds)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Overwriting driver.py


Let's test the module.

In [5]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [6]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [7]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

'count key/values in json.  possible duplicate: length of javascript associative array   i have a json that looks like this:  this prints when i do this:  i am trying to get the count of the items inside this json, obj.length returns "undefined" and obj[0].length returns   uncaught typeerror: cannot read property \'length\' of undefined  i would expect a length to return "2" in this case. how can i find the count? thanks!'

Here, we define a helper function to convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [8]:
def text_to_json(text):
    return json.dumps({'input':'{0}'.format(text)})

In [9]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [10]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 588.71 ms


In [11]:
resp = predict_for(body)

INFO:model_driver:Predictions: [[(5223, 6700, 0.9964429348386941), (11922383, 11922384, 0.9608198958942987), (14220321, 14220323, 0.23205773009236366), (4616202, 4616273, 0.19269681609181566), (20279484, 20279485, 0.15604203701328218), (85992, 86014, 0.14896617620396632), (19590865, 19590901, 0.05871890364999471), (126100, 4889658, 0.05265433128981724), (4255472, 4255480, 0.04450552792725994), (750486, 750506, 0.019287321340651926), (122102, 122704, 0.011277641639918021), (12953704, 12953750, 0.008323144663828217), (3127429, 3127440, 0.006957983908504305), (2631001, 2631198, 0.0056324526343852835), (4968406, 4968448, 0.005262096049088963), (4935632, 4935684, 0.005133128628901672), (7364150, 7364307, 0.003974935184494744), (6491463, 6491621, 0.0037887065235385174), (1129216, 1129270, 0.00201454162412491), (901115, 901144, 0.0018603601973077998), (1069666, 1069840, 0.0017272926196244432), (695050, 695053, 0.0016822210756380301), (728360, 728694, 0.001574943657350216), (684672, 684692, 0.

Next, we move on to building our docker image.